In [1]:
from fastapi import FastAPI, File, UploadFile, HTTPException
import torch
import torch.nn as nn
from torchvision import transforms
from PIL import Image
from torchvision import transforms
import io

In [3]:
app = FastAPI()
app

In [4]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # Matches the training architecture
        self.fc2 = nn.Linear(128, 10)

    def forward(self, feature):
        feature = F.relu(self.conv1(feature))
        feature = self.pool(feature)
        feature = F.relu(self.conv2(feature))
        feature = self.pool(feature)
        feature = feature.view(feature.size(0), -1)  # Flatten
        feature = F.relu(self.fc1(feature))
        feature = self.fc2(feature)
        return feature

cnn_model = CNNModel()

# Load the saved state dictionary
cnn_model.load_state_dict(torch.load('../model/(CNN)digit_recognition_model.pth', map_location=torch.device('cpu'),weights_only=True))

cnn_model.eval()

CNNModel(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [5]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [7]:
@app.post("/predict/")
async def predict_digit(file: UploadFile = File(...)):
    try:
        # Read the image data from the uploaded file
        image_data = await file.read()
        image = Image.open(io.BytesIO(image_data))

        # Preprocess the image
        image = transform(image)  # Apply transformations
        image = image.unsqueeze(0)  # Add batch dimension [1, 1, 28, 28]

        # Get the prediction
        with torch.no_grad():
            output = cnn_model(image)
            _, predicted = torch.max(output, 1)
            predicted_digit = predicted.item()

        return {"predicted_digit": predicted_digit}

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Internal Server Error: {str(e)}")


In [ ]:
import nest_asyncio
nest_asyncio.apply()

import uvicorn
uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [52145]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:52861 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:52861 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:52861 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:52861 - "GET /openapi.json HTTP/1.1" 200 OK


/Users/visheshpanghal/Downloads/Data_Science/deep-learning/Handwritten_Digit_Recognition/lib/python3.12/site-packages/fastapi/openapi/utils.py:225: UserWarning: Duplicate Operation ID predict_digit_predict__post for function predict_digit
  warnings.warn(message, stacklevel=1)


INFO:     127.0.0.1:52862 - "POST /predict/ HTTP/1.1" 500 Internal Server Error
